<a href="https://colab.research.google.com/github/ananyashreyjain/colab/blob/master/SOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Hyper paramaters

test_size = 0.2
random_state = 42
oov_token = '<oov>'
num_words = 1e7
embedding_dim = 100
trunc_type = 'post'
padding_type = 'post'
maxlen = 100

In [1]:
!wget --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz -O /content/clothes.json.gz
!wget --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz -O /content/sports.json.gz
!wget --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz -O /content/phones.json.gz
!wget --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz -O /content/games.json.gz

--2020-04-01 17:11:46--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47285200 (45M) [application/x-gzip]
Saving to: ‘/content/clothes.json.gz’

/content/clothes.js 100%[===================>]  45.09M  4.51MB/s    in 13s     

2020-04-01 17:12:00 (3.34 MB/s) - ‘/content/clothes.json.gz’ saved [47285200/47285200]

--2020-04-01 17:12:02--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68275834 (65M) [application/x-gzip]
Saving to: ‘/content/sports.json.gz’

/content/sports.j

In [0]:
!mkdir gdrive
from google.colab import drive
drive.mount('/gdrive')

mkdir: cannot create directory ‘gdrive’: File exists
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!gunzip clothes.json.gz
!gunzip sports.json.gz
!gunzip games.json.gz
!gunzip phones.json.gz

In [0]:
import pandas as pd
data_clothes = pd.read_json('clothes.json', lines=True)
data_sports = pd.read_json('sports.json', lines=True)
data_games = pd.read_json('games.json', lines=True)
data_phones = pd.read_json('phones.json', lines=True)

In [4]:
print(data_clothes.shape)
print(data_sports.shape)
print(data_games.shape)
print(data_phones.shape)

(278677, 9)
(296337, 9)
(167597, 9)
(194439, 9)


In [5]:
data_clothes.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5,Grammy's Angels Love it,1394841600,"03 15, 2014"


In [0]:
X_clothes, Y_clothes = data_clothes['reviewText'], data_clothes['overall']
X_sports, Y_sports = data_sports['reviewText'], data_sports['overall']
X_games, Y_games = data_games['reviewText'], data_games['overall']
X_phones, Y_phones = data_phones['reviewText'], data_phones['overall']

0         This is a great tutu and at a really great pri...
1         I bought this for my 4 yr old daughter for dan...
2         What can I say... my daughters have it in oran...
3         We bought several tutus at once, and they are ...
4         Thank you Halo Heaven great product for Little...
                                ...                        
278672    I don't normally go ga-ga over a product very ...
278673    I've been traveling back and forth to England ...
278674    These are very nice packing cubes and the 18 x...
278675    I am on vacation with my family of four and th...
278676    When I signed up to receive a free set of Shac...
Name: reviewText, Length: 278677, dtype: object

In [0]:
X_train = [None]*4
X_test = [None]*4
Y_train = [None]*4
Y_test = [None]*4
from sklearn.model_selection import train_test_split
X_train[0], X_test[0], Y_train[0], Y_test[0] = train_test_split(X_clothes, Y_clothes, random_state=42, test_size=test_size, shuffle=True)
X_train[1], X_test[1], Y_train[1], Y_test[1] = train_test_split(X_sports, Y_sports, random_state=42, test_size=test_size, shuffle=True)
X_train[2], X_test[2], Y_train[2], Y_test[2] = train_test_split(X_games, Y_games, random_state=42, test_size=test_size, shuffle=True)
X_train[3], X_test[3], Y_train[3], Y_test[3] = train_test_split(X_phones, Y_phones, random_state=42, test_size=test_size, shuffle=True)

In [70]:
print(X_train[0].shape, Y_train[0].shape)
print(X_test[0].shape, Y_test[0].shape)

(222941,) (222941,)
(55736,) (55736,)


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences_train =[None]*4
sequences_test = [None]*4
padded_train = [None]*4
padded_test = [None]*4
for _ in range(0,4):
  tokenizer[_] = Tokenizer(num_words=num_words)
  tokenizer[_].fit_on_texts(X_train[_])
  sequences_train[_] = tokenizer[_].texts_to_sequences(X_train[_])
  sequences_test[_] = tokenizer[_].texts_to_sequences(X_test[_])
  padded_train[_] = pad_sequences(sequences_train[_], maxlen=maxlen, padding=padding_type, truncating=trunc_type)
  padded_test[_] = pad_sequences(sequences_test[_], maxlen=maxlen, padding=padding_type, truncating=trunc_type)


In [89]:
vocab_size = [len(tokenizer[0].index_word),len(tokenizer[1].index_word),len(tokenizer[2].index_word),len(tokenizer[3].index_word)]
print(vocab_size)

[75949, 119637, 80762, 81436]


In [109]:
print(X_train[0].iloc[1])
string = [tokenizer[0].index_word.get(x," ") for x in padded_train[0][1]]
print(string)
print(len(string))

I LOVE THE COLORS OF THESE SOCKS BUT ONE COLOR COBINATION ;ONE OF THE PRETTIEST--TURQUIISE AND YELLOW IS WAY TOO TIGHT TO PULL THEM ON WITHOUT A CROWBAR! THE OTHERS FIT FINE. ILL BUY MORE IF YOU CAN MAKE THEM BIGGER. THE COLORS ARE WONDERFUL!
['i', 'love', 'the', 'colors', 'of', 'these', 'socks', 'but', 'one', 'color', 'cobination', 'one', 'of', 'the', 'prettiest', 'turquiise', 'and', 'yellow', 'is', 'way', 'too', 'tight', 'to', 'pull', 'them', 'on', 'without', 'a', 'crowbar', 'the', 'others', 'fit', 'fine', 'ill', 'buy', 'more', 'if', 'you', 'can', 'make', 'them', 'bigger', 'the', 'colors', 'are', 'wonderful', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
100


In [112]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size[0], embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='spare_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          7594900   
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 64)            32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 24, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 7,660,053
Trainable params: 7,660,053
Non-trainable params: 0
______________________________________________

In [113]:
num_epochs = 1
history = model.fit(X_train[0], Y_train[0], epochs=num_epochs, validation_data=(X_test[1], Y_test[1]), verbose=2)

ERROR! Session/line number was not unique in database. History logging moved to new session 61


ValueError: ignored